In [1]:
#imports
import numpy as np
import pandas as pd

import re
import time

from bs4 import BeautifulSoup
import requests

In [2]:
start_time = time.time()

page_n = 1 #keep track of which page the scraper is on
games = []# list to collect data
base_url = 'https://boardgamegeek.com/' #the base url
start_url = 'https://boardgamegeek.com/browse/boardgame/page/' #start page

# a list of attributes to be collected from 'all_d'
collect_list = ["objectid","name","yearpublished","sortindex","minplayers",
                "maxplayers","minplaytime","maxplaytime","minage","best",
                "max","totalvotes","playerage","languagedependence", 
                "usersrated","average","baverage","stddev","avgweight", 
                "numweights","numgeeklists","numtrading","numwanting",
                "numcomments","views","numplays","numplays_month","news",
                "blogs","weblink","podcast","label","boardgamedesigner",
                "boardgameartist","boardgamepublisher","boardgamehonor",
                "boardgamecategory","boardgamemechanic","boardgameexpansion",
                "boardgameversion","boardgamefamily"]
#a list of attributes to be collected from "credit_d"
credit_collect_list = ["boardgamedesigner","boardgameartist","boardgamepublisher",
                       "boardgamehonor","boardgamecategory","boardgameversion"]
# while loop to continue until page_n reached the decided limit
while page_n <= 200 :
    #request the page
    game_names = BeautifulSoup(requests.get(start_url+str(page_n)).text,'lxml')
    #get list of games from page
    game_list = game_names.find_all('td',{'class':'collection_objectname'}) 
    
    #below is logic to stop the loop if no additional links are available
    if not game_list:
        print('No more results: Exiting...') 
        break
    else:
        print('Getting page:{}'.format(page_n))
        
        #iterate through game list for page_n
        for game in game_list:
            game_data = [] #collect data
            #request game page
            game_page = BeautifulSoup(requests.get(base_url+\
                        game.select('a')[0]['href']+'/stats').text,'html.parser')
            #capture whole description of game
            desc_tag = game_page.find_all('meta',{'name':'description'})[0]
            desc_str = str(desc_tag)[15:-29].replace('&amp;ldquo;','"')\
            .replace('&amp;rdquo;','"').replace('\n',' ')
            
            #manipulate js script data and create dictionaries for attribute look up
            script =game_page.find("script", text=re.compile("GEEK.geekitemPreload\s+="))
            data_list = str(script).replace('=4&mt=8&at=','').replace(':{"link"','')\
            .split("=",9)[9].strip().split(':{')
            
            comm_d = {i.split(':')[0]:' '.join(i.split(':')[1:])\
                      for i in data_list[3][:-1].split(',') if len(i.split(':'))> 1 }
            stats_d = {i.split(':')[0]:' '.join(i.split(':')[1:])\
                       for i in data_list[5][:-1].split(',') if len(i.split(':'))> 1 }
            counts_d = {i.split(':')[0]:' '.join(i.split(':')[1:])\
                        for i in data_list[6][:-1].split(',') if len(i.split(':'))> 1 }
            info_d = {i.split(':')[0]:' '.join(i.split(':')[1:])\
                      for i in data_list[7][:-1].split(',') if len(i.split(':'))> 1 }
            cr_cnt_d = {i.split(':')[0]:' '.join(i.split(':')[1:])\
                        for i in data_list[9][:-1].split(',') if len(i.split(':'))> 1 }
            rank_d = {i.split(':')[0]:' '.join(i.split(':')[1:])\
                      for i in data_list[10][:-1].split(',') if len(i.split(':'))> 1 }
            
            all_d = {**comm_d, **stats_d,**counts_d,**info_d,**cr_cnt_d,**rank_d}
            
            #extract data from messy string
            credit_d = {}
            for i in data_list[8].split('],'):
                item = i.split(":",1)
                try:
                    values = [h.split(',')[0] for h in\
                              [j.split(':')[1] for j in item[1].split('},{')]]
                except:
                    pass
                credit_d[item[0]]=values
                
            #iterate through list of desired attributes to capture values
            for i in collect_list:
                if '"'+i+'"' in all_d.keys():
                    game_data.append(all_d['"'+i+'"'].replace('"',''))
                else:
                    game_data.append('NaN')
            for i in credit_collect_list:
                if '"'+i+'"' in credit_d.keys():
                    game_data.append(credit_d['"'+i+'"'])
                else:
                    game_data.append('NaN')
            
            game_data.append(desc_str) #append game description
            games.append(game_data)
        page_n += 1
        
end_time = time.time()

print('total_time: {} minutes'.format((end_time-start_time)/60))

Getting page:1
Getting page:2
Getting page:3
Getting page:4
Getting page:5
Getting page:6
Getting page:7
Getting page:8
Getting page:9
Getting page:10
Getting page:11
Getting page:12
Getting page:13
Getting page:14
Getting page:15
Getting page:16
Getting page:17
Getting page:18
Getting page:19
Getting page:20
Getting page:21
Getting page:22
Getting page:23
Getting page:24
Getting page:25
Getting page:26
Getting page:27
Getting page:28
Getting page:29
Getting page:30
Getting page:31
Getting page:32
Getting page:33
Getting page:34
Getting page:35
Getting page:36
Getting page:37
Getting page:38
Getting page:39
Getting page:40
Getting page:41
Getting page:42
Getting page:43
Getting page:44
Getting page:45
Getting page:46
Getting page:47
Getting page:48
Getting page:49
Getting page:50
Getting page:51
Getting page:52
Getting page:53
Getting page:54
Getting page:55
Getting page:56
Getting page:57
Getting page:58
Getting page:59
Getting page:60
Getting page:61
Getting page:62
Getting page:63
G

IndexError: list index out of range

In [4]:
t_game = BeautifulSoup(requests.get(base_url+\
                        game.select('a')[0]['href']+'/stats').text,'html.parser')

In [19]:
desc_tag = t_game.find_all('meta')

In [21]:
desc_tag[9]

<meta content="https://boardgamegeek.com/boardgame/22203/face-mat" property="og:url">
<meta content="website" property="og:type">
<meta content="Highly detailed wrestling die and chart simulation. The game contains hundreds of wrestler cards that depict fictional and historical wrestlers.  The game was further supported by additional 'wrestler packs' that add to the game.  Tournament or league play.  For enthusiasts of the sport primarily." property="og:description">
<meta content="summary_large_image" name="twitter:card"/>
<meta content="Face To The Mat" name="twitter:title"/>
<meta content="https://cf.geekdo-images.com/opengraph/img/-D5RbF9hQjqWTM9A8ieyL1WxxBY=/fit-in/1200x630/pic586029.jpg" name="twitter:image:src"/>
<meta content="board game, boardgames, boardgame, board, games, game, hobby, boardgamegeek, geek, geekdo" name="keywords"/>
<link href="https://cf.geekdo-static.com/static/geekui_master2_5e1665a53dbf7.css" rel="stylesheet" type="text/css"/>
<base href="/"/>
<script cros

In [5]:
df = pd.DataFrame.from_records(games)

In [7]:
df

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,174430,Gloomhaven,2017,1,1,4,60,120,12,[{min 3,...,4,19,7,"[""Isaac Childres""]","[""Alexandr Elichev"", ""Josh T. McDowell"", ""Alva...","[""Cephalofair Games"", ""Albi"", ""Asmodee"", ""Feue...","[""2017 Best Science Fiction or Fantasy Board G...","[""Adventure"", ""Exploration"", ""Fantasy"", ""Fight...","[""Chinese edition"", ""Czech edition"", ""English ...",Gloomhaven is a game of Euro-inspired tactica...
1,161936,Pandemic Legacy Season 1,2015,1,2,4,60,60,13,[{min 4,...,0,33,3,"[""Rob Daviau"", ""Matt Leacock""]","[""Chris Quilliams""]","[""Z-Man Games, ""Asterion Press"", ""Devir"", ""Fil...","[""2015 Cardboard Republic Immersionist Laurel ...","[""Environmental"", ""Medical""]","[""Chinese blue edition"", ""Chinese red edition""...",Pandemic Legacy is a co-operative campaign gam...
2,167791,Terraforming Mars,2016,1,1,5,120,120,12,[{min 3,...,15,29,6,"[""Jacob Fryxelius""]","[""Isaac Fryxelius""]","[""FryxGames"", ""Arclight"", ""Fantasmagoria"", ""Gh...","[""2016 Cardboard Republic Architect Laurel Nom...","[""Economic"", ""Environmental"", ""Industry \/ Man...","[""Bulgarian edition"", ""Chinese edition"", ""Czec...","In the 2400s, mankind begins to terraform the ..."
3,182028,Through the Ages A New Story of Civilization,2015,1,2,4,120,120,14,[{min 3,...,1,14,2,"[""Vlaada Chv\u00e1til""]","[""Filip Murmak"", ""Radim Pech"", ""Jakub Politzer...","[""Czech Games Edition"", ""Cranio Creations"", ""D...","[""2015 Golden Geek Best Strategy Board Game No...","[""Card Game"", ""Civilization"", ""Economic""]","[""Chinese edition"", ""Czech edition"", ""English ...",Through the Ages: A New Story of Civilization ...
4,224517,Brass Birmingham,2018,1,2,4,60,120,14,[{min 3,...,0,9,6,"[""Gavan Brown"", ""Matt Tolman"", ""Martin Wallace""]","[""Lina Cossette"", ""David Forest"", ""Damien Mamm...","[""Roxley"", ""BoardM Factory"", ""Conclave Editora...","[""2018 Golden Geek Best Board Game Artwork & P...","[""Economic"", ""Industry \/ Manufacturing"", ""Tra...","[""English deluxe edition"", ""English retail edi...",Brass: Birmingham is an economic strategy game...
5,233078,Twilight Imperium (Fourth Edition),2017,1,3,6,240,480,14,[{min 6,...,0,11,3,"[""Dane Beltrami"", ""Corey Konieczka"", ""Christia...","[""Scott Schomburg""]","[""Fantasy Flight Games"", ""ADC Blackfire Entert...","[""2017 Golden Geek Best Strategy Board Game No...","[""Civilization"", ""Economic"", ""Negotiation"", ""P...","[""Chinese edition"", ""Czech edition"", ""English ...",Twilight Imperium (Fourth Edition) is a game o...
6,12333,Twilight Struggle,2005,1,2,2,120,180,13,[{min 2,...,9,32,5,"[""Ananda Gupta"", ""Jason Matthews""]","[""Viktor Csete"", ""Rodger B. MacGowan"", ""Chechu...","[""GMT Games"", ""(Self-Published)"", ""Asterion Pr...","[""2005 Charles S. Roberts Best Modern Era Boar...","[""Modern Warfare"", ""Political"", ""Wargame""]","[""Bard Centrum Gier Polish deluxe edition"", ""B...","&amp;quot;Now the trumpet summons us again, no..."
7,187645,Star Wars Rebellion,2016,1,2,4,180,240,14,[{min 2,...,1,10,2,"[""Corey Konieczka""]","[""Matt Allsopp"", ""David Ardila"", ""Balaskas"", ""...","[""Fantasy Flight Games"", ""ADC Blackfire Entert...","[""2016 Best Science Fiction or Fantasy Board G...","[""Fighting"", ""Miniatures"", ""Movies \/ TV \/ Ra...","[""Czech edition"", ""English edition"", ""French e...",From the publisher: Star Wars: Rebellion is a...
8,220308,Gaia Project,2017,1,1,4,60,150,12,[{min 3,...,0,13,3,"[""Jens Dr\u00f6gem\u00fcller"", ""Helge Ostertag""]","[""Dennis Lohausen""]","[""Feuerland Spiele"", ""Cranio Creations"", ""Dice...","[""2017 Golden Geek Best Solo Board Game Nomine...","[""Civilization"", ""Economic"", ""Science Fiction""...","[""Chinese edition"", ""Dutch edition"", ""English ...",Gaia Project is a new game in the line of Terr...
9,169786,Scythe,2016,1,1,5,90,115,14,[{min 4,...,16,21,6,"[""Jamey Stegmaier""]","[""Jakub Rozalski""]","[""Stonemaier Games"", ""Albi""